# Using jupyter-jchannel in a local notebook

## Installing

You can use your favorite manager to install the [PyPI package](https://pypi.org/project/jupyter-jchannel/).

```
pip install jupyter-jchannel
```

## Starting a server

For basic usage, you only need to import the `jchannel` module.

In [1]:
import jchannel

A [jchannel Server](https://jupyter-jchannel.readthedocs.io/en/latest/jchannel.server.html#jchannel.server.Server) instance runs *alongside* a Jupyter server instead of *over* it. Therefore, it needs its own local address.

In [2]:
HOST = '127.0.0.1'
PORT = 8889

The asynchronous [start function](https://jupyter-jchannel.readthedocs.io/en/latest/jchannel.html#jchannel.start) instantiates a server, starts this server and returns it.

In [ ]:
server = await jchannel.start(host=HOST, port=PORT)

## Opening a channel

A [server channel](https://jupyter-jchannel.readthedocs.io/en/latest/jchannel.channel.html#jchannel.channel.Channel) uses a server to call frontend JavaScript code from kernel Python code. The asynchronous [open method](https://jupyter-jchannel.readthedocs.io/en/latest/jchannel.server.html#jchannel.server.Server.open) instantiates a channel, opens this channel and returns it.

This method receives a string representing a JavaScript function. This function should receive a [client representation of the same channel](https://hashiprobr.github.io/jupyter-jchannel-client/Channel.html) and initialize it. The most important part of this initialization is setting the `handler` property to an object. The methods of this object define the API at the frontend.

The example below sets `handler` to an object with a single method called `indent`. This method uses the JavaScript [padStart method](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/String/padStart) to add blank spaces at the beginning of a string.

In [4]:
channel = await server.open('''
    (channel) => {
        channel.handler = {
            indent(line, count) {
                return line.padStart(line.length + count);
            }
        };
    }
''')

And that's it! You can now call any method of the API from the notebook.

The `call` method returns an `asyncio.Task` that can be awaited for the result.

Arguments and return values can have any types that are JSON-serializable.

In [5]:
await channel.call('indent', 'return', 4)

'    return'

If the JavaScript method throws an exception, it is wrapped in a Python exception.

In [6]:
await channel.call('indent', 4, 'return')

FrontendError: TypeError: line.padStart is not a function

Calling kernel code from frontend code is even simpler, as seen in [this tutorial](local-frontend.ipynb).